#### Notes
1. Importing Spreadsheets or CSV files (QGIS3) - https://www.qgistutorials.com/en/docs/3/importing_spreadsheets_csv.html
2. Nearest Neighbor Analysis (QGIS3) - https://www.qgistutorials.com/en/docs/3/nearest_neighbor_analysis.html
3. Getting list of coordinates (lat,long) from OSMNX Geoseries - https://copyprogramming.com/howto/plotting-a-route-with-osmnx
4. QNEAT3 Algorithms - https://root676.github.io/OdMatrixAlgs
5. Source Paper - https://www.researchgate.net/publication/319291962_The_Traditional_Four_Steps_Transportation_Modeling_Using_Simplified_Transport_Network_A_Case_Study_of_Dhaka_City_Bangladesh

In [1]:
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Steps in Generating Demand Matrix

In [2]:
# CSV File generated from QGIS
df_centroids_to_node = pd.read_csv("final_barangay_centroids_to_nearest_node.csv")
df_centroids_to_node_rerun = pd.read_csv("final_rerun_barangay_centroids_to_nearest_node.csv")

# Rename columns
df_centroids_to_node_rerun.rename(columns={'HubName':'Barangay Name','field_1':'HubName','lat':'Latitude', 'lon':'Longitude'}, inplace=True)
df_centroids_to_node_rerun.drop(columns=["number"], inplace=True)
df_centroids_to_node_final = pd.concat([df_centroids_to_node, df_centroids_to_node_rerun])

df_cn = df_centroids_to_node_final[["Barangay Name","HubName"]]
#df_cn.set_index('Barangay Name', inplace=True)
df_cn_final = df_cn.groupby(['HubName'])['Barangay Name'].apply(','.join).reset_index()
df_cn_final.rename(columns={'HubName':'Node','Barangay Name':'Centroids'}, inplace=True)
df_cn_final.set_index("Node", inplace=True)

df_cn_final

,Centroids
Node,
11th Ave,"Pineda, Pasig"
15th Ave,"E. Rodriguez, Quezon,East Kamias, Quezon,San R..."
5th Ave 1,"CBarangay 103, Caloocan,CBarangay 104, Calooca..."
5th Ave 2,"MBarangay 127, Manila,MBarangay 128, Manila,NB..."
A Mabini,"CBarangay 13, Caloocan,CBarangay 15, Caloocan,..."
...,...
Victoneta Ave,"CBarangay 132, Caloocan,CBarangay 142, Calooca..."
Visayas Ave,"Pasong Tamo, Quezon"
Waltermart Sucat,"San Isidro, Paranaque"


#### 9 Modal Split
#### 9.1 Utility Matrices 

Centroid and then to Node: Cost Matrix Tables (in terms of time and cost) using QGIS

Network Type: https://www.qgistutorials.com/en/docs/3/downloading_osm_data.html

Vector -> QuickOSM -> QuickOSM dialog box

Processing -> Toolbox -> QNEAT3 -> Distance Matrices -> OD-Matrix From Points as Table (n:n)

#### 9.1.a. Bus

'type'='route' and 'public_transport:version'='2'

In [ ]:
# Travel Time Matrix, Travel Cost Matrix
# Bus, Car, Motorcycle

# bus: 15 pesos for the first 5 km, 2 pesos for every succeeding km

In [3]:
csv_bus_travel_dist_mat = pd.read_csv("final_bus_cost_matrix.csv")
csv_bus_travel_dist_mat.head()

,origin_id,destination_id,entry_cost,network_cost,exit_cost,total_cost
0,"CBarangay 1, Caloocan","CBarangay 1, Caloocan",0.000000,0.000000,0.000000,0.000000
1,"CBarangay 1, Caloocan","CBarangay 10, Caloocan",42.697224,829.118859,54.724247,926.540330
2,"CBarangay 1, Caloocan","CBarangay 100, Caloocan",42.697224,2790.148200,146.895781,2979.741206
3,"CBarangay 1, Caloocan","CBarangay 101, Caloocan",42.697224,2632.708475,302.933888,2978.339587
4,"CBarangay 1, Caloocan","CBarangay 102, Caloocan",42.697224,8048.832226,187.487586,8279.017036


In [4]:
len(csv_bus_travel_dist_mat)

2924100

In [5]:
# Convert to a Matrix
df_bus_travel_dist_mat = csv_bus_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,"CBarangay 1, Caloocan","CBarangay 1, Caloocan",0.000000
1,"CBarangay 1, Caloocan","CBarangay 10, Caloocan",926.540330
2,"CBarangay 1, Caloocan","CBarangay 100, Caloocan",2979.741206
3,"CBarangay 1, Caloocan","CBarangay 101, Caloocan",2978.339587
4,"CBarangay 1, Caloocan","CBarangay 102, Caloocan",8279.017036


In [6]:
# assign centroid to a centroid_number
list_cen = df_bus_travel_dist_mat["origin_id"].unique()
df_cen_no = pd.DataFrame(list_cen, columns=["Centroid"])
cen_no = list(range(0,len(list_cen)))
df_cen_no["Number"] = cen_no
df_cen_no.set_index("Centroid", inplace=True)
df_cen_no.head()

,Number
Centroid,
"CBarangay 1, Caloocan",0
"CBarangay 10, Caloocan",1
"CBarangay 100, Caloocan",2
"CBarangay 101, Caloocan",3
"CBarangay 102, Caloocan",4


In [7]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

In [8]:
df_bus_travel_dist_mat["origin_id"] = df_bus_travel_dist_mat["origin_id"].apply(cen_number)
df_bus_travel_dist_mat["destination_id"] = df_bus_travel_dist_mat["destination_id"].apply(cen_number)

In [14]:
df_bus_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,926.540330
2,0,2,2979.741206
3,0,3,2978.339587
4,0,4,8279.017036


In [15]:
rows = df_bus_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_bus_travel_dist_mat)):
    i = df_bus_travel_dist_mat["origin_id"][a]
    j = df_bus_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_bus_travel_dist_mat["total_cost"][a]

#print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

In [16]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,926.540330,2979.741206,2978.339587,8279.017036,2835.894813,2680.281353,3222.342290,2135.601730,2216.683316,...,8638.870474,7972.042456,9637.257090,8449.447431,13785.085041,6215.580069,10558.019645,8761.276422,8035.330898,11721.489771
1,926.540330,0.000000,3507.380681,3505.979063,8806.656511,3363.534288,3198.681764,3504.791571,2663.241205,2744.322792,...,8618.469248,7951.641230,9616.855863,8429.046205,13764.683815,6195.178843,10537.618419,9288.915897,8014.929671,11701.088545
2,2979.741206,3507.380681,0.000000,1744.134324,7906.739564,1601.689549,1446.076090,1988.137027,1824.127688,1446.645198,...,7350.015394,9606.392163,11271.606797,10083.797138,11821.152346,7849.929776,12192.369352,7289.105520,9669.680605,13355.839478
3,2978.339587,3505.979063,1744.134324,0.000000,7844.391384,463.423000,934.247841,849.870478,1312.299439,934.816949,...,8800.358156,9712.806177,11378.020810,10190.211151,13271.495108,7956.343789,12298.783365,8739.448282,9776.094618,13462.253491
4,8279.017036,8806.656511,7906.739564,7844.391384,0.000000,7393.424079,7682.624467,7532.561403,7804.686408,7885.767995,...,7064.192577,12492.428043,14157.642676,12969.833017,12228.375918,10735.965656,15078.405232,7001.228684,12555.716484,16241.875357


In [ ]:
#for g in range(0, len(df_cost_mat)):
#    df_cost_mat.loc[g,g] = 300

#df_cost_mat

In [17]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_bus_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_bus_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,2.223697,7.151379,7.148015,19.869641,6.806148,6.432675,7.733621,5.125444,5.320040,...,20.733289,19.132902,23.129417,20.278674,33.084204,14.917392,25.339247,21.027063,19.284794,28.131575
1,2.223697,0.000000,8.417714,8.414350,21.135976,8.072482,7.676836,8.411500,6.391779,6.586375,...,20.684326,19.083939,23.080454,20.229711,33.035241,14.868429,25.290284,22.293398,19.235831,28.082613
2,7.151379,8.417714,0.000000,4.185922,18.976175,3.844055,3.470583,4.771529,4.377906,3.471948,...,17.640037,23.055341,27.051856,24.201113,28.370766,18.839831,29.261686,17.493853,23.207233,32.054015
3,7.148015,8.414350,4.185922,0.000000,18.826539,1.112215,2.242195,2.039689,3.149519,2.243561,...,21.120860,23.310735,27.307250,24.456507,31.851588,19.095225,29.517080,20.974676,23.462627,32.309408
4,19.869641,21.135976,18.976175,18.826539,0.000000,17.744218,18.438299,18.078147,18.731247,18.925843,...,16.954062,29.981827,33.978342,31.127599,29.348102,25.766318,36.188173,16.802949,30.133720,38.980501


In [35]:
# Number of NAN values
df_bus_travel_time_mat.isna().sum().sum()

160500

In [36]:
# Get the maximum value and fill the missing values
max_times = df_bus_travel_time_mat.max()
max_time = max_times.max()
max_time

144.96555260879998

In [37]:
df_bus_travel_time_mat = df_bus_travel_time_mat.fillna(150)
df_bus_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,2.223697,7.151379,7.148015,19.869641,6.806148,6.432675,7.733621,5.125444,5.320040,...,20.733289,19.132902,23.129417,20.278674,33.084204,14.917392,25.339247,21.027063,19.284794,28.131575
1,2.223697,0.000000,8.417714,8.414350,21.135976,8.072482,7.676836,8.411500,6.391779,6.586375,...,20.684326,19.083939,23.080454,20.229711,33.035241,14.868429,25.290284,22.293398,19.235831,28.082613
2,7.151379,8.417714,0.000000,4.185922,18.976175,3.844055,3.470583,4.771529,4.377906,3.471948,...,17.640037,23.055341,27.051856,24.201113,28.370766,18.839831,29.261686,17.493853,23.207233,32.054015
3,7.148015,8.414350,4.185922,0.000000,18.826539,1.112215,2.242195,2.039689,3.149519,2.243561,...,21.120860,23.310735,27.307250,24.456507,31.851588,19.095225,29.517080,20.974676,23.462627,32.309408
4,19.869641,21.135976,18.976175,18.826539,0.000000,17.744218,18.438299,18.078147,18.731247,18.925843,...,16.954062,29.981827,33.978342,31.127599,29.348102,25.766318,36.188173,16.802949,30.133720,38.980501


In [38]:
df_bus_travel_time_mat.isna().sum().sum()

0

In [18]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

first_five = 15  
every_km = 2    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        if (dist <= 5):
            rate = first_five
        else:
            rate = first_five + every_km * (dist - 5) 
        
        final_cost_mat[i][j] = round(rate,2)

#print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

In [19]:
df_bus_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_bus_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,15.00,15.00,15.00,15.00,21.56,15.00,15.00,15.00,15.00,15.00,...,22.28,20.94,24.27,21.90,32.57,17.43,26.12,22.52,21.07,28.44
1,15.00,15.00,15.00,15.00,22.61,15.00,15.00,15.00,15.00,15.00,...,22.24,20.90,24.23,21.86,32.53,17.39,26.08,23.58,21.03,28.40
2,15.00,15.00,15.00,15.00,20.81,15.00,15.00,15.00,15.00,15.00,...,19.70,24.21,27.54,25.17,28.64,20.70,29.38,19.58,24.34,31.71
3,15.00,15.00,15.00,15.00,20.69,15.00,15.00,15.00,15.00,15.00,...,22.60,24.43,27.76,25.38,31.54,20.91,29.60,22.48,24.55,31.92
4,21.56,22.61,20.81,20.69,15.00,19.79,20.37,20.07,20.61,20.77,...,19.13,29.98,33.32,30.94,29.46,26.47,35.16,19.00,30.11,37.48


In [21]:
# Get the number of NAN values
df_bus_travel_cost_mat.isna().sum().sum()

160500

In [22]:
# Get the maximum value and fill the missing values
max_costs = df_bus_travel_cost_mat.max()
max_cost = max_costs.max()
max_cost

125.8

In [23]:
df_bus_travel_cost_mat = df_bus_travel_cost_mat.fillna(150)
df_bus_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,15.00,15.00,15.00,15.00,21.56,15.00,15.00,15.00,15.00,15.00,...,22.28,20.94,24.27,21.90,32.57,17.43,26.12,22.52,21.07,28.44
1,15.00,15.00,15.00,15.00,22.61,15.00,15.00,15.00,15.00,15.00,...,22.24,20.90,24.23,21.86,32.53,17.39,26.08,23.58,21.03,28.40
2,15.00,15.00,15.00,15.00,20.81,15.00,15.00,15.00,15.00,15.00,...,19.70,24.21,27.54,25.17,28.64,20.70,29.38,19.58,24.34,31.71
3,15.00,15.00,15.00,15.00,20.69,15.00,15.00,15.00,15.00,15.00,...,22.60,24.43,27.76,25.38,31.54,20.91,29.60,22.48,24.55,31.92
4,21.56,22.61,20.81,20.69,15.00,19.79,20.37,20.07,20.61,20.77,...,19.13,29.98,33.32,30.94,29.46,26.47,35.16,19.00,30.11,37.48


In [24]:
df_bus_travel_cost_mat.isna().sum().sum()

0

In [25]:
df_cost = df_cn_final["Centroids"].to_frame()
list_c = df_cost["Centroids"].to_list()

list_cn = []
for c in list_c:
    ccn = []
    cens = c.replace(', ','*').split(',')
    for d in cens:
        d = d.replace('*',', ')
        cn = cen_number(d)
        ccn.append(cn)
    list_cn.append(ccn)
        
df_cost["Centroid_Numbers"] = list_cn
df_cost.head()

,Centroids,Centroid_Numbers
Node,,
11th Ave,"Pineda, Pasig",[1461]
15th Ave,"E. Rodriguez, Quezon,East Kamias, Quezon,San R...","[1521, 1522, 1593, 1607]"
5th Ave 1,"CBarangay 103, Caloocan,CBarangay 104, Calooca...","[5, 6, 7, 11, 13, 14, 15, 16, 17, 18, 19, 20, ..."
5th Ave 2,"MBarangay 127, Manila,MBarangay 128, Manila,NB...","[317, 318, 1218, 1219, 1222]"
A Mabini,"CBarangay 13, Caloocan,CBarangay 15, Caloocan,...","[34, 56, 67, 78, 89, 99, 101, 102, 103, 104, 1..."


In [26]:
# Import df_nodes (final_bus_nodes.csv)
df_nodes = pd.read_csv("final_bus_nodes.csv")
df_nodes.set_index("Unnamed: 0", inplace=True)
df_nodes.head()

,number,lat,lon
Unnamed: 0,,,
Monumento,1,14.654455,120.983795
Malanday,2,14.718282,120.957038
Mall of Asia,3,14.535539,120.983720
PITX,4,14.510395,120.992016
North EDSA,5,14.654670,121.029643


In [27]:
df_nodes["node_number"] = range(0,len(df_nodes))
df_nodes.drop(columns="number", inplace=True)
df_nodes

,lat,lon,node_number
Unnamed: 0,,,
Monumento,14.654455,120.983795,0
Malanday,14.718282,120.957038,1
Mall of Asia,14.535539,120.983720,2
PITX,14.510395,120.992016,3
North EDSA,14.654670,121.029643,4
...,...,...,...
Calle Industria,14.605401,121.078977,226
Tiendesitas,14.585557,121.078620,227
Arnaiz Ave,14.550995,121.011371,228


In [28]:
len(df_nodes)

231

In [29]:
list_node_numbers = []
list_indices = list(df_cost.index)
for i in list_indices:
    num = df_nodes["node_number"][i]
    list_node_numbers.append(num)
    
df_cost["Node_Number"] = list_node_numbers
df_cost.head()

,Centroids,Centroid_Numbers,Node_Number
Node,,,
11th Ave,"Pineda, Pasig",[1461],221
15th Ave,"E. Rodriguez, Quezon,East Kamias, Quezon,San R...","[1521, 1522, 1593, 1607]",103
5th Ave 1,"CBarangay 103, Caloocan,CBarangay 104, Calooca...","[5, 6, 7, 11, 13, 14, 15, 16, 17, 18, 19, 20, ...",33
5th Ave 2,"MBarangay 127, Manila,MBarangay 128, Manila,NB...","[317, 318, 1218, 1219, 1222]",38
A Mabini,"CBarangay 13, Caloocan,CBarangay 15, Caloocan,...","[34, 56, 67, 78, 89, 99, 101, 102, 103, 104, 1...",34


In [31]:
df_nn_cn = df_cost[["Node_Number","Centroid_Numbers"]].set_index("Node_Number")
df_nn_cn

,Centroid_Numbers
Node_Number,
221,[1461]
103,"[1521, 1522, 1593, 1607]"
33,"[5, 6, 7, 11, 13, 14, 15, 16, 17, 18, 19, 20, ..."
38,"[317, 318, 1218, 1219, 1222]"
34,"[34, 56, 67, 78, 89, 99, 101, 102, 103, 104, 1..."
...,...
25,"[37, 48, 169, 170, 255]"
163,[1570]
182,[1239]


In [32]:
no_of_nodes = len(df_nn_cn.index)
no_of_nodes

231

In [39]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_bus_travel_time_mat[a][b]
                sum_cost = sum_cost + df_bus_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

#print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
#print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

In [40]:
bus_travel_time_mat = pd.DataFrame(final_node_time_mat)
bus_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.95,26.41,32.57,42.08,14.38,42.92,20.80,52.31,29.99,36.24,...,150.0,35.46,62.57,18.40,22.99,57.38,39.40,34.60,22.13,26.13
1,26.41,88.60,55.94,65.45,35.56,43.67,43.68,53.08,53.36,59.61,...,150.0,58.64,79.41,36.35,38.55,80.42,61.41,57.95,42.50,43.31
2,32.57,55.94,0.00,11.67,41.35,70.53,34.38,79.92,5.11,8.62,...,150.0,15.75,50.50,42.21,48.61,35.75,35.04,9.80,40.01,46.34
3,42.08,65.45,11.67,2.83,50.69,80.03,43.71,89.43,13.77,10.35,...,150.0,21.88,56.85,51.54,57.00,39.43,38.55,15.22,47.99,54.67
4,14.38,35.56,41.35,50.69,2.80,39.41,15.60,48.81,38.66,44.05,...,150.0,37.70,67.34,10.72,17.93,52.23,30.33,43.14,14.38,18.45


In [41]:
bus_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
bus_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,15.00,27.01,32.14,40.07,17.00,40.76,22.33,48.60,29.99,35.20,...,150.0,34.55,62.60,20.34,24.16,52.82,37.84,33.83,23.44,26.78
1,27.01,90.38,51.62,59.54,34.63,41.40,41.40,49.23,49.47,54.67,...,150.0,53.86,76.63,35.29,37.13,72.02,56.18,53.29,40.42,41.09
2,32.14,51.62,15.00,15.71,39.46,63.77,33.65,71.60,15.00,15.09,...,150.0,18.12,52.61,40.17,45.51,34.79,34.20,15.02,38.34,43.62
3,40.07,59.54,15.71,15.00,47.24,71.70,41.43,79.53,16.52,15.20,...,150.0,23.23,57.83,47.95,52.50,37.86,37.12,17.69,44.99,50.56
4,17.00,34.63,39.46,47.24,15.00,37.84,18.00,45.68,37.22,41.71,...,150.0,36.42,66.57,15.05,19.94,48.53,30.28,40.95,17.11,20.38


In [43]:
bus_travel_time_mat.isna().sum().sum(), bus_travel_cost_mat.isna().sum().sum()

(0, 0)

In [44]:
# Export as .csv
bus_travel_time_mat = bus_travel_time_mat.to_csv("bus_travel_time_mat.csv")
bus_travel_cost_mat = bus_travel_cost_mat.to_csv("bus_travel_cost_mat.csv")

#### 9.1.b. Car

'highway'='primary' OR 'highway'='secondary' OR 'highway'='tertiary' OR 'route'='road'

In [45]:
csv_car_travel_dist_mat = pd.read_csv("final_car_cost_matrix.csv")
#csv_car_travel_dist_mat.head()

# Convert to a Matrix
df_car_travel_dist_mat = csv_car_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_car_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,"CBarangay 1, Caloocan","CBarangay 1, Caloocan",0.000000
1,"CBarangay 1, Caloocan","CBarangay 10, Caloocan",11334.994966
2,"CBarangay 1, Caloocan","CBarangay 100, Caloocan",12558.108578
3,"CBarangay 1, Caloocan","CBarangay 101, Caloocan",11635.769616
4,"CBarangay 1, Caloocan","CBarangay 102, Caloocan",74062.081851


In [46]:
df_cen_no.head()

,Number
Centroid,
"CBarangay 1, Caloocan",0
"CBarangay 10, Caloocan",1
"CBarangay 100, Caloocan",2
"CBarangay 101, Caloocan",3
"CBarangay 102, Caloocan",4


In [47]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

df_car_travel_dist_mat["origin_id"] = df_car_travel_dist_mat["origin_id"].apply(cen_number)
df_car_travel_dist_mat["destination_id"] = df_car_travel_dist_mat["destination_id"].apply(cen_number)

df_car_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,11334.994966
2,0,2,12558.108578
3,0,3,11635.769616
4,0,4,74062.081851


In [48]:
rows = df_car_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_car_travel_dist_mat)):
    i = df_car_travel_dist_mat["origin_id"][a]
    j = df_car_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_car_travel_dist_mat["total_cost"][a]

#print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

In [49]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,11334.994966,12558.108578,11635.769616,74062.081851,11493.324841,11337.711382,12081.615219,10793.031759,10874.113345,...,8008.707471,8981.020678,10047.411457,10286.797671,12542.939743,7507.712129,13290.456426,10625.435902,8326.642927,13764.521924
1,11334.994966,0.000000,4306.707910,2772.843914,64473.952105,2630.399139,2474.785680,3218.689517,1930.106056,2011.187643,...,8568.310885,7226.622678,6739.652569,6689.703497,12750.557141,5753.314129,9909.944937,7974.128883,7728.839189,10384.010434
2,12558.108578,4306.707910,0.000000,2141.558439,63715.277763,1690.591134,1981.965052,1312.125502,2538.950701,2468.680945,...,9184.994239,10020.549920,10261.831489,10211.882418,13510.637737,8547.241372,13432.123858,6632.599634,10147.705850,13906.189355
3,11635.769616,2772.843914,2141.558439,0.000000,63799.418552,463.423000,934.247841,1053.540046,1312.299439,934.816949,...,9180.043620,9098.210959,9276.088243,9226.139172,13714.275892,7624.902410,12446.380612,8079.739166,9225.366888,12920.446109
4,74062.081851,64473.952105,63715.277763,63799.418552,0.000000,63348.451246,63634.960286,63152.419345,63430.101214,63827.790397,...,71606.355855,71524.523193,71104.156180,71054.207109,76140.588127,70051.214644,74274.448549,69920.814068,71651.679123,74748.514046


In [50]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_car_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_car_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,27.203988,30.139461,27.925847,177.748996,27.583980,27.210507,28.995877,25.903276,26.097872,...,19.220898,21.554450,24.113787,24.688314,30.103055,18.018509,31.897095,25.501046,19.983943,33.034853
1,27.203988,0.000000,10.336099,6.654825,154.737485,6.312958,5.939486,7.724855,4.632255,4.826850,...,20.563946,17.343894,16.175166,16.055288,30.601337,13.807954,23.783868,19.137909,18.549214,24.921625
2,30.139461,10.336099,0.000000,5.139740,152.916667,4.057419,4.756716,3.149101,6.093482,5.924834,...,22.043986,24.049320,24.628396,24.508518,32.425531,20.513379,32.237097,15.918239,24.354494,33.374854
3,27.925847,6.654825,5.139740,0.000000,153.118605,1.112215,2.242195,2.528496,3.149519,2.243561,...,22.032105,21.835706,22.262612,22.142734,32.914262,18.299766,29.871313,19.391374,22.140881,31.009071
4,177.748996,154.737485,152.916667,153.118605,0.000000,152.036283,152.723905,151.565806,152.232243,153.186697,...,171.855254,171.658856,170.649975,170.530097,182.737412,168.122915,178.258677,167.809954,171.964030,179.396434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,18.018509,13.807954,20.513379,18.299766,168.122915,17.957898,17.584426,19.369795,16.277195,16.471791,...,10.624410,3.551450,6.110787,6.827170,19.486967,0.000000,13.894095,15.885048,7.434844,15.031853
1706,31.897095,23.783868,32.237097,29.871313,178.258677,29.529446,29.155974,30.941343,27.848743,28.043338,...,24.291032,10.347793,7.799263,8.341039,27.492262,13.894095,0.000000,29.763634,15.440139,4.112021
1707,25.501046,19.137909,15.918239,19.391374,167.809954,18.950706,18.794260,18.042388,17.487029,17.681625,...,6.605658,19.420988,21.980326,22.554853,16.987202,15.885048,29.763634,0.000000,16.113507,30.901391
1708,19.983943,18.549214,24.354494,22.140881,171.964030,21.799013,21.425541,23.210910,20.118310,20.312905,...,9.553033,5.097493,7.656831,8.373213,12.754263,7.434844,15.440139,16.113507,0.000000,16.577896


In [51]:
# Get the number of NAN values
df_car_travel_time_mat.isna().sum().sum()

20

In [52]:
# Get the maximum value and fill the missing values
max_times = df_car_travel_time_mat.max()
max_time = max_times.max()
max_time

196.91325821928

In [53]:
df_car_travel_time_mat = df_car_travel_time_mat.fillna(220)
df_car_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,27.203988,30.139461,27.925847,177.748996,27.583980,27.210507,28.995877,25.903276,26.097872,...,19.220898,21.554450,24.113787,24.688314,30.103055,18.018509,31.897095,25.501046,19.983943,33.034853
1,27.203988,0.000000,10.336099,6.654825,154.737485,6.312958,5.939486,7.724855,4.632255,4.826850,...,20.563946,17.343894,16.175166,16.055288,30.601337,13.807954,23.783868,19.137909,18.549214,24.921625
2,30.139461,10.336099,0.000000,5.139740,152.916667,4.057419,4.756716,3.149101,6.093482,5.924834,...,22.043986,24.049320,24.628396,24.508518,32.425531,20.513379,32.237097,15.918239,24.354494,33.374854
3,27.925847,6.654825,5.139740,0.000000,153.118605,1.112215,2.242195,2.528496,3.149519,2.243561,...,22.032105,21.835706,22.262612,22.142734,32.914262,18.299766,29.871313,19.391374,22.140881,31.009071
4,177.748996,154.737485,152.916667,153.118605,0.000000,152.036283,152.723905,151.565806,152.232243,153.186697,...,171.855254,171.658856,170.649975,170.530097,182.737412,168.122915,178.258677,167.809954,171.964030,179.396434


In [54]:
# Get the number of NAN values
df_car_travel_time_mat.isna().sum().sum()

0

In [55]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

base_fare = 40  
every_km = 15    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        rate = base_fare + every_km * dist
        
        final_cost_mat[i][j] = round(rate,2)

#print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

In [56]:
df_car_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_car_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,40.00,210.02,228.37,214.54,1150.93,212.40,210.07,221.22,201.90,203.11,...,160.13,174.72,190.71,194.30,228.14,152.62,239.36,199.38,164.90,246.47
1,210.02,40.00,104.60,81.59,1007.11,79.46,77.12,88.28,68.95,70.17,...,168.52,148.40,141.09,140.35,231.26,126.30,188.65,159.61,155.93,195.76
2,228.37,104.60,40.00,72.12,995.73,65.36,69.73,59.68,78.08,77.03,...,177.77,190.31,193.93,193.18,242.66,168.21,241.48,139.49,192.22,248.59
3,214.54,81.59,72.12,40.00,996.99,46.95,54.01,55.80,59.68,54.02,...,177.70,176.47,179.14,178.39,245.71,154.37,226.70,161.20,178.38,233.81
4,1150.93,1007.11,995.73,996.99,40.00,990.23,994.52,987.29,991.45,997.42,...,1114.10,1112.87,1106.56,1105.81,1182.11,1090.77,1154.12,1088.81,1114.78,1161.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,152.62,126.30,168.21,154.37,1090.77,152.24,149.90,161.06,141.73,142.95,...,106.40,62.20,78.19,82.67,161.79,40.00,126.84,139.28,86.47,133.95
1706,239.36,188.65,241.48,226.70,1154.12,224.56,222.22,233.38,214.05,215.27,...,191.82,104.67,88.75,92.13,211.83,126.84,40.00,226.02,136.50,65.70
1707,199.38,159.61,139.49,161.20,1088.81,158.44,157.46,152.76,149.29,150.51,...,81.29,161.38,177.38,180.97,146.17,139.28,226.02,40.00,140.71,233.13
1708,164.90,155.93,192.22,178.38,1114.78,176.24,173.91,185.07,165.74,166.96,...,99.71,71.86,87.86,92.33,119.71,86.47,136.50,140.71,40.00,143.61


In [57]:
# Get the number of NAN values
df_car_travel_cost_mat.isna().sum().sum()

20

In [58]:
# Get the maximum value and fill the missing values
max_costs = df_car_travel_cost_mat.max()
max_cost = max_costs.max()
max_cost

1270.71

In [59]:
df_car_travel_cost_mat = df_car_travel_cost_mat.fillna(1350)
df_car_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,40.00,210.02,228.37,214.54,1150.93,212.40,210.07,221.22,201.90,203.11,...,160.13,174.72,190.71,194.30,228.14,152.62,239.36,199.38,164.90,246.47
1,210.02,40.00,104.60,81.59,1007.11,79.46,77.12,88.28,68.95,70.17,...,168.52,148.40,141.09,140.35,231.26,126.30,188.65,159.61,155.93,195.76
2,228.37,104.60,40.00,72.12,995.73,65.36,69.73,59.68,78.08,77.03,...,177.77,190.31,193.93,193.18,242.66,168.21,241.48,139.49,192.22,248.59
3,214.54,81.59,72.12,40.00,996.99,46.95,54.01,55.80,59.68,54.02,...,177.70,176.47,179.14,178.39,245.71,154.37,226.70,161.20,178.38,233.81
4,1150.93,1007.11,995.73,996.99,40.00,990.23,994.52,987.29,991.45,997.42,...,1114.10,1112.87,1106.56,1105.81,1182.11,1090.77,1154.12,1088.81,1114.78,1161.23


In [60]:
# Get the number of NAN values
df_car_travel_cost_mat.isna().sum().sum()

0

In [61]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_car_travel_time_mat[a][b]
                sum_cost = sum_cost + df_car_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

#print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
#print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

In [62]:
car_travel_time_mat = pd.DataFrame(final_node_time_mat)
car_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.89,24.47,32.64,41.74,13.93,40.04,18.41,47.11,29.93,34.97,...,37.83,33.40,27.60,18.46,22.83,33.41,39.37,33.51,22.29,27.71
1,24.47,10.95,54.02,63.13,35.47,38.76,38.57,46.09,51.84,56.88,...,58.31,55.28,49.66,35.14,37.50,53.46,60.10,55.55,41.05,42.53
2,32.64,54.02,0.00,10.98,34.77,68.11,33.25,75.18,5.12,7.74,...,25.73,15.75,13.61,40.92,47.19,32.35,34.44,8.93,38.64,46.47
3,41.74,63.13,10.98,1.48,43.87,77.21,42.35,84.27,13.51,9.90,...,29.38,21.38,21.94,50.02,56.29,36.39,38.27,12.86,47.47,52.88
4,13.93,35.47,34.77,43.87,1.68,40.83,5.84,47.90,31.59,36.56,...,28.69,29.81,26.42,12.07,18.34,21.77,28.41,33.51,12.10,18.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,33.41,53.46,32.35,36.39,21.77,45.98,16.24,52.19,28.59,28.11,...,13.34,18.35,22.93,21.12,23.12,0.00,9.20,26.78,14.89,17.22
227,39.37,60.10,34.44,38.27,28.41,53.55,22.88,59.76,30.68,29.99,...,11.64,20.21,25.00,28.75,30.71,9.20,0.00,28.67,22.52,24.81
228,33.51,55.55,8.93,12.86,33.51,67.71,31.94,74.77,5.14,4.77,...,19.78,10.44,10.01,39.66,45.93,26.78,28.67,1.58,37.23,43.27
229,22.29,41.05,38.64,47.47,12.10,37.34,8.87,43.69,35.45,38.97,...,25.13,29.13,29.22,7.48,11.66,14.89,22.52,37.23,2.04,9.93


In [63]:
car_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
car_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,58.06,192.97,244.02,300.88,127.07,290.28,155.04,334.43,227.06,258.58,...,276.42,248.76,212.52,155.36,182.72,248.83,286.05,249.47,179.32,213.17
1,192.97,108.46,377.66,434.54,261.71,282.24,281.05,328.06,363.98,395.50,...,404.46,385.48,350.38,259.64,274.38,374.12,415.61,387.20,296.59,305.79
2,244.02,377.66,40.00,108.64,257.34,465.70,247.82,509.86,71.98,88.35,...,200.82,138.46,125.08,295.75,334.94,242.22,255.28,95.81,281.50,330.42
3,300.88,434.54,108.64,49.27,314.20,522.56,304.68,566.71,124.44,101.89,...,223.62,173.60,177.15,352.60,391.80,267.40,279.20,120.37,336.67,370.48
4,127.07,261.71,257.34,314.20,50.50,295.20,76.52,339.36,237.46,268.51,...,219.31,226.32,205.15,115.43,154.62,176.06,217.54,249.45,115.62,156.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,248.83,374.12,242.22,267.40,176.06,327.36,141.53,366.22,218.70,215.66,...,123.39,154.70,183.30,172.02,184.51,40.00,97.52,207.39,133.07,147.63
227,286.05,415.61,255.28,279.20,217.54,374.66,183.01,413.51,231.76,227.44,...,112.74,166.30,196.27,219.71,231.96,97.52,40.00,219.18,180.76,195.08
228,249.47,387.20,95.81,120.37,249.45,463.16,239.61,507.31,72.10,69.80,...,163.61,105.26,102.53,287.85,327.04,207.39,219.18,49.86,272.71,310.46
229,179.32,296.59,281.50,336.67,115.62,273.35,95.41,313.09,261.56,283.56,...,197.07,222.08,222.64,86.77,112.86,133.07,180.76,272.71,52.73,102.09


In [65]:
# Export as .csv
car_travel_time_mat = car_travel_time_mat.to_csv("car_travel_time_mat.csv")
car_travel_cost_mat = car_travel_cost_mat.to_csv("car_travel_cost_mat.csv")

#### 9.1.c. Motorcycle

'highway'='primary' OR 'highway'='secondary' OR 'highway'='tertiary' OR 'highway'='motorway' OR 'route'='road'

OR 'highway'='primary_link' OR 'highway'='secondary_link' OR 'highway'='tertiary_link' OR 'highway'='motorway_link'

In [101]:
csv_motor_travel_dist_mat = pd.read_csv("final_car_cost_matrix.csv")
#csv_motor_travel_dist_mat.head()

# Convert to a Matrix
df_motor_travel_dist_mat = csv_motor_travel_dist_mat.drop(columns=["entry_cost","network_cost","exit_cost"])
df_motor_travel_dist_mat.head()

,origin_id,destination_id,total_cost
0,"CBarangay 1, Caloocan","CBarangay 1, Caloocan",0.000000
1,"CBarangay 1, Caloocan","CBarangay 10, Caloocan",11334.994966
2,"CBarangay 1, Caloocan","CBarangay 100, Caloocan",12558.108578
3,"CBarangay 1, Caloocan","CBarangay 101, Caloocan",11635.769616
4,"CBarangay 1, Caloocan","CBarangay 102, Caloocan",74062.081851


In [67]:
df_cen_no.head()

,Number
Centroid,
"CBarangay 1, Caloocan",0
"CBarangay 10, Caloocan",1
"CBarangay 100, Caloocan",2
"CBarangay 101, Caloocan",3
"CBarangay 102, Caloocan",4


In [103]:
def cen_number(centroid):
    number = df_cen_no["Number"][centroid]
    return number

df_motor_travel_dist_mat["origin_id"] = df_motor_travel_dist_mat["origin_id"].apply(cen_number)
df_motor_travel_dist_mat["destination_id"] = df_motor_travel_dist_mat["destination_id"].apply(cen_number)

df_motor_travel_dist_mat.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2852\3309001711.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number = df_cen_no["Number"][centroid]


,origin_id,destination_id,total_cost
0,0,0,0.000000
1,0,1,11334.994966
2,0,2,12558.108578
3,0,3,11635.769616
4,0,4,74062.081851


In [104]:
rows = df_motor_travel_dist_mat["origin_id"].nunique()
cols = rows
 
dist_mat = [[0 for _ in range(cols)] for _ in range(rows)]

for a in range(0, len(df_motor_travel_dist_mat)):
    i = df_motor_travel_dist_mat["origin_id"][a]
    j = df_motor_travel_dist_mat["destination_id"][a]
    dist_mat[i][j] = df_motor_travel_dist_mat["total_cost"][a]

#print(f'matrix of dimension {rows} x {cols} is {dist_mat}')

In [105]:
df_dist_mat = pd.DataFrame(dist_mat)
df_dist_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,11334.994966,12558.108578,11635.769616,74062.081851,11493.324841,11337.711382,12081.615219,10793.031759,10874.113345,...,8008.707471,8981.020678,10047.411457,10286.797671,12542.939743,7507.712129,13290.456426,10625.435902,8326.642927,13764.521924
1,11334.994966,0.000000,4306.707910,2772.843914,64473.952105,2630.399139,2474.785680,3218.689517,1930.106056,2011.187643,...,8568.310885,7226.622678,6739.652569,6689.703497,12750.557141,5753.314129,9909.944937,7974.128883,7728.839189,10384.010434
2,12558.108578,4306.707910,0.000000,2141.558439,63715.277763,1690.591134,1981.965052,1312.125502,2538.950701,2468.680945,...,9184.994239,10020.549920,10261.831489,10211.882418,13510.637737,8547.241372,13432.123858,6632.599634,10147.705850,13906.189355
3,11635.769616,2772.843914,2141.558439,0.000000,63799.418552,463.423000,934.247841,1053.540046,1312.299439,934.816949,...,9180.043620,9098.210959,9276.088243,9226.139172,13714.275892,7624.902410,12446.380612,8079.739166,9225.366888,12920.446109
4,74062.081851,64473.952105,63715.277763,63799.418552,0.000000,63348.451246,63634.960286,63152.419345,63430.101214,63827.790397,...,71606.355855,71524.523193,71104.156180,71054.207109,76140.588127,70051.214644,74274.448549,69920.814068,71651.679123,74748.514046


In [106]:
# Convert from total_cost (distance) to minute

def dist_to_min(dist):
    fixed_rate = 25    # kph
    dist = dist/1000   # convert to km
    min = (dist/fixed_rate)*60
    return min

df_motor_travel_time_mat = pd.DataFrame(df_dist_mat.apply(dist_to_min))
df_motor_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,27.203988,30.139461,27.925847,177.748996,27.583980,27.210507,28.995877,25.903276,26.097872,...,19.220898,21.554450,24.113787,24.688314,30.103055,18.018509,31.897095,25.501046,19.983943,33.034853
1,27.203988,0.000000,10.336099,6.654825,154.737485,6.312958,5.939486,7.724855,4.632255,4.826850,...,20.563946,17.343894,16.175166,16.055288,30.601337,13.807954,23.783868,19.137909,18.549214,24.921625
2,30.139461,10.336099,0.000000,5.139740,152.916667,4.057419,4.756716,3.149101,6.093482,5.924834,...,22.043986,24.049320,24.628396,24.508518,32.425531,20.513379,32.237097,15.918239,24.354494,33.374854
3,27.925847,6.654825,5.139740,0.000000,153.118605,1.112215,2.242195,2.528496,3.149519,2.243561,...,22.032105,21.835706,22.262612,22.142734,32.914262,18.299766,29.871313,19.391374,22.140881,31.009071
4,177.748996,154.737485,152.916667,153.118605,0.000000,152.036283,152.723905,151.565806,152.232243,153.186697,...,171.855254,171.658856,170.649975,170.530097,182.737412,168.122915,178.258677,167.809954,171.964030,179.396434


In [107]:
# Get the number of NAN values
df_motor_travel_time_mat.isna().sum().sum()

20

In [108]:
# Get the maximum value and fill the missing values
max_times = df_motor_travel_time_mat.max()
max_time = max_times.max()
max_time

196.91325821928

In [109]:
df_motor_travel_time_mat = df_motor_travel_time_mat.fillna(220)
df_motor_travel_time_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,0.000000,27.203988,30.139461,27.925847,177.748996,27.583980,27.210507,28.995877,25.903276,26.097872,...,19.220898,21.554450,24.113787,24.688314,30.103055,18.018509,31.897095,25.501046,19.983943,33.034853
1,27.203988,0.000000,10.336099,6.654825,154.737485,6.312958,5.939486,7.724855,4.632255,4.826850,...,20.563946,17.343894,16.175166,16.055288,30.601337,13.807954,23.783868,19.137909,18.549214,24.921625
2,30.139461,10.336099,0.000000,5.139740,152.916667,4.057419,4.756716,3.149101,6.093482,5.924834,...,22.043986,24.049320,24.628396,24.508518,32.425531,20.513379,32.237097,15.918239,24.354494,33.374854
3,27.925847,6.654825,5.139740,0.000000,153.118605,1.112215,2.242195,2.528496,3.149519,2.243561,...,22.032105,21.835706,22.262612,22.142734,32.914262,18.299766,29.871313,19.391374,22.140881,31.009071
4,177.748996,154.737485,152.916667,153.118605,0.000000,152.036283,152.723905,151.565806,152.232243,153.186697,...,171.855254,171.658856,170.649975,170.530097,182.737412,168.122915,178.258677,167.809954,171.964030,179.396434


In [110]:
# Get the number of NAN values
df_motor_travel_time_mat.isna().sum().sum()

0

In [111]:
no_of_centroids = len(df_dist_mat.index)
r = no_of_centroids
c = no_of_centroids

first_two = 50
every_km = 10    
     
final_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        dist = df_dist_mat[i][j]
        dist = dist/1000     # convert to km

        if (dist <= 2):
            rate = first_two
        else:
            rate = first_two + every_km * (dist - 2) 
        
        final_cost_mat[i][j] = round(rate,2)

#print(f'matrix of dimension {r} x {c} is {final_cost_mat}')

In [112]:
df_motor_travel_cost_mat = pd.DataFrame(final_cost_mat)
df_motor_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,50.00,143.35,155.58,146.36,770.62,144.93,143.38,150.82,137.93,138.74,...,110.09,119.81,130.47,132.87,155.43,105.08,162.90,136.25,113.27,167.65
1,143.35,50.00,73.07,57.73,674.74,56.30,54.75,62.19,50.00,50.11,...,115.68,102.27,97.40,96.90,157.51,87.53,129.10,109.74,107.29,133.84
2,155.58,73.07,50.00,51.42,667.15,50.00,50.00,50.00,55.39,54.69,...,121.85,130.21,132.62,132.12,165.11,115.47,164.32,96.33,131.48,169.06
3,146.36,57.73,51.42,50.00,667.99,50.00,50.00,50.00,50.00,50.00,...,121.80,120.98,122.76,122.26,167.14,106.25,154.46,110.80,122.25,159.20
4,770.62,674.74,667.15,667.99,50.00,663.48,666.35,661.52,664.30,668.28,...,746.06,745.25,741.04,740.54,791.41,730.51,772.74,729.21,746.52,777.49


In [113]:
# Get the number of NAN values
df_motor_travel_cost_mat.isna().sum().sum()

20

In [114]:
# Get the maximum value and fill the missing values
max_costs = df_motor_travel_cost_mat.max()
max_cost = max_costs.max()
max_cost

850.47

In [115]:
df_motor_travel_cost_mat = df_motor_travel_cost_mat.fillna(900)
df_motor_travel_cost_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709
0,50.00,143.35,155.58,146.36,770.62,144.93,143.38,150.82,137.93,138.74,...,110.09,119.81,130.47,132.87,155.43,105.08,162.90,136.25,113.27,167.65
1,143.35,50.00,73.07,57.73,674.74,56.30,54.75,62.19,50.00,50.11,...,115.68,102.27,97.40,96.90,157.51,87.53,129.10,109.74,107.29,133.84
2,155.58,73.07,50.00,51.42,667.15,50.00,50.00,50.00,55.39,54.69,...,121.85,130.21,132.62,132.12,165.11,115.47,164.32,96.33,131.48,169.06
3,146.36,57.73,51.42,50.00,667.99,50.00,50.00,50.00,50.00,50.00,...,121.80,120.98,122.76,122.26,167.14,106.25,154.46,110.80,122.25,159.20
4,770.62,674.74,667.15,667.99,50.00,663.48,666.35,661.52,664.30,668.28,...,746.06,745.25,741.04,740.54,791.41,730.51,772.74,729.21,746.52,777.49


In [116]:
# Get the number of NAN values
df_motor_travel_cost_mat.isna().sum().sum()

0

In [117]:
r = no_of_nodes
c = no_of_nodes
 
final_node_time_mat = [[0 for _ in range(c)] for _ in range(r)]
final_node_cost_mat = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        list_i = df_nn_cn["Centroid_Numbers"][i]
        list_j = df_nn_cn["Centroid_Numbers"][j]

        sum_cost = 0
        sum_time = 0
        counter = 0
        for a in list_i:
            for b in list_j:
                sum_time = sum_time + df_motor_travel_time_mat[a][b]
                sum_cost = sum_cost + df_motor_travel_cost_mat[a][b]
                counter = counter + 1
        
        final_node_time_mat[i][j] = round(sum_time/counter,2)
        final_node_cost_mat[i][j] = round(sum_cost/counter,2)

#print(f'travel time matrix of dimension {r} x {c} is {final_node_time_mat}')
#print(f'travel cost matrix of dimension {r} x {c} is {final_node_cost_mat}')

In [118]:
motor_travel_time_mat = pd.DataFrame(final_node_time_mat)
motor_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.89,24.47,32.64,41.74,13.93,40.04,18.41,47.11,29.93,34.97,...,37.83,33.40,27.60,18.46,22.83,33.41,39.37,33.51,22.29,27.71
1,24.47,10.95,54.02,63.13,35.47,38.76,38.57,46.09,51.84,56.88,...,58.31,55.28,49.66,35.14,37.50,53.46,60.10,55.55,41.05,42.53
2,32.64,54.02,0.00,10.98,34.77,68.11,33.25,75.18,5.12,7.74,...,25.73,15.75,13.61,40.92,47.19,32.35,34.44,8.93,38.64,46.47
3,41.74,63.13,10.98,1.48,43.87,77.21,42.35,84.27,13.51,9.90,...,29.38,21.38,21.94,50.02,56.29,36.39,38.27,12.86,47.47,52.88
4,13.93,35.47,34.77,43.87,1.68,40.83,5.84,47.90,31.59,36.56,...,28.69,29.81,26.42,12.07,18.34,21.77,28.41,33.51,12.10,18.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,33.41,53.46,32.35,36.39,21.77,45.98,16.24,52.19,28.59,28.11,...,13.34,18.35,22.93,21.12,23.12,0.00,9.20,26.78,14.89,17.22
227,39.37,60.10,34.44,38.27,28.41,53.55,22.88,59.76,30.68,29.99,...,11.64,20.21,25.00,28.75,30.71,9.20,0.00,28.67,22.52,24.81
228,33.51,55.55,8.93,12.86,33.51,67.71,31.94,74.77,5.14,4.77,...,19.78,10.44,10.01,39.66,45.93,26.78,28.67,1.58,37.23,43.27
229,22.29,41.05,38.64,47.47,12.10,37.34,8.87,43.69,35.45,38.97,...,25.13,29.13,29.22,7.48,11.66,14.89,22.52,37.23,2.04,9.93


In [119]:
motor_travel_cost_mat = pd.DataFrame(final_node_cost_mat)
motor_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,50.15,131.98,166.02,203.92,88.05,196.85,106.69,226.29,154.71,175.72,...,187.61,169.17,145.01,106.90,125.14,169.22,194.04,169.64,122.88,145.45
1,131.98,79.44,255.10,293.02,177.81,191.49,190.70,222.04,245.98,267.00,...,272.98,260.32,236.92,176.42,186.25,252.74,280.41,261.47,201.06,207.19
2,166.02,255.10,50.00,75.76,174.89,313.80,168.55,343.24,51.77,62.23,...,137.21,95.64,86.72,200.50,226.62,164.81,173.52,67.21,191.00,223.61
3,203.92,293.02,75.76,50.00,212.80,351.71,206.45,381.14,86.29,71.26,...,152.41,119.06,121.43,238.40,264.53,181.61,189.46,83.58,227.78,250.32
4,88.05,177.81,174.89,212.80,50.00,200.14,54.35,229.57,161.64,182.34,...,149.54,154.21,140.10,80.28,106.41,120.71,148.36,169.63,80.41,107.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,169.22,252.74,164.81,181.61,120.71,221.57,97.69,247.48,149.14,147.11,...,85.59,106.46,125.53,118.01,126.34,50.00,68.35,141.59,92.04,101.75
227,194.04,280.41,173.52,189.46,148.36,253.10,125.34,279.01,157.84,154.96,...,78.49,114.20,134.18,149.81,157.98,68.35,50.00,149.45,123.84,133.39
228,169.64,261.47,67.21,83.58,169.63,312.11,163.07,341.54,52.45,52.99,...,112.40,73.51,71.69,195.24,221.36,141.59,149.45,50.00,185.14,210.31
229,122.88,201.06,191.00,227.78,80.41,185.57,66.94,212.05,177.71,192.38,...,134.71,151.39,151.76,61.18,78.57,92.04,123.84,185.14,50.00,71.39


In [120]:
# Export as .csv
motor_travel_time_mat = motor_travel_time_mat.to_csv("motor_travel_time_mat.csv")
motor_travel_cost_mat = motor_travel_cost_mat.to_csv("motor_travel_cost_mat.csv")

#### 9.1.d. Utility Functions

In [121]:
def U_car(travel_time, travel_cost):
    u_car = 0.345505 -0.060054*travel_time - 0.043648*travel_cost
    return u_car

def U_bus(travel_time, travel_cost):
    u_bus = 0.645505 -0.060054*travel_time - 0.043648*travel_cost
    return u_bus

def U_motor(travel_time, travel_cost):
    u_motor = 1.23213 -0.060054*travel_time - 0.043648*travel_cost
    return u_motor

In [122]:
# Import 
bus_travel_time_mat = pd.read_csv("bus_travel_time_mat.csv")
bus_travel_cost_mat = pd.read_csv("bus_travel_cost_mat.csv")
car_travel_time_mat = pd.read_csv("car_travel_time_mat.csv")
car_travel_cost_mat = pd.read_csv("car_travel_cost_mat.csv")
motor_travel_time_mat = pd.read_csv("motor_travel_time_mat.csv")
motor_travel_cost_mat = pd.read_csv("motor_travel_cost_mat.csv")

In [123]:
bus_travel_time_mat.drop(columns="Unnamed: 0", inplace=True)
bus_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.95,26.41,32.57,42.08,14.38,42.92,20.80,52.31,29.99,36.24,...,150.0,35.46,62.57,18.40,22.99,57.38,39.40,34.60,22.13,26.13
1,26.41,88.60,55.94,65.45,35.56,43.67,43.68,53.08,53.36,59.61,...,150.0,58.64,79.41,36.35,38.55,80.42,61.41,57.95,42.50,43.31
2,32.57,55.94,0.00,11.67,41.35,70.53,34.38,79.92,5.11,8.62,...,150.0,15.75,50.50,42.21,48.61,35.75,35.04,9.80,40.01,46.34
3,42.08,65.45,11.67,2.83,50.69,80.03,43.71,89.43,13.77,10.35,...,150.0,21.88,56.85,51.54,57.00,39.43,38.55,15.22,47.99,54.67
4,14.38,35.56,41.35,50.69,2.80,39.41,15.60,48.81,38.66,44.05,...,150.0,37.70,67.34,10.72,17.93,52.23,30.33,43.14,14.38,18.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,57.38,80.42,35.75,39.43,52.23,80.36,45.05,89.76,32.05,32.07,...,150.0,25.70,62.17,51.67,55.35,0.00,30.34,35.32,47.25,50.21
227,39.40,61.41,35.04,38.55,30.33,57.61,23.15,65.60,31.35,31.20,...,150.0,24.00,60.38,29.77,29.86,30.34,0.00,34.60,23.51,24.72
228,34.60,57.95,9.80,15.22,43.14,72.41,35.89,81.81,5.95,8.92,...,150.0,15.32,50.35,43.73,50.13,35.32,34.60,3.62,41.52,47.86
229,22.13,42.50,40.01,47.99,14.38,38.59,9.85,47.64,37.10,40.31,...,150.0,32.72,64.61,10.08,10.94,47.25,23.51,41.52,2.51,8.43


In [124]:
bus_travel_cost_mat.drop(columns="Unnamed: 0", inplace=True)
bus_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,15.00,27.01,32.14,40.07,17.00,40.76,22.33,48.60,29.99,35.20,...,150.0,34.55,62.60,20.34,24.16,52.82,37.84,33.83,23.44,26.78
1,27.01,90.38,51.62,59.54,34.63,41.40,41.40,49.23,49.47,54.67,...,150.0,53.86,76.63,35.29,37.13,72.02,56.18,53.29,40.42,41.09
2,32.14,51.62,15.00,15.71,39.46,63.77,33.65,71.60,15.00,15.09,...,150.0,18.12,52.61,40.17,45.51,34.79,34.20,15.02,38.34,43.62
3,40.07,59.54,15.71,15.00,47.24,71.70,41.43,79.53,16.52,15.20,...,150.0,23.23,57.83,47.95,52.50,37.86,37.12,17.69,44.99,50.56
4,17.00,34.63,39.46,47.24,15.00,37.84,18.00,45.68,37.22,41.71,...,150.0,36.42,66.57,15.05,19.94,48.53,30.28,40.95,17.11,20.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,52.82,72.02,34.79,37.86,48.53,71.96,42.54,79.80,31.71,31.73,...,150.0,26.42,62.27,48.06,51.12,15.00,30.28,34.43,44.37,46.84
227,37.84,56.18,34.20,37.12,30.28,53.01,24.29,59.67,31.12,31.00,...,150.0,25.00,60.77,29.81,29.88,30.28,15.00,33.84,24.59,25.60
228,33.83,53.29,15.02,17.69,40.95,65.34,34.91,73.17,15.00,15.32,...,150.0,17.89,52.77,41.44,46.77,34.43,33.84,15.00,39.60,44.88
229,23.44,40.42,38.34,44.99,17.11,37.16,15.00,44.70,35.92,38.59,...,150.0,32.27,64.29,15.00,15.00,44.37,24.59,39.60,15.00,15.00


In [125]:
car_travel_time_mat.drop(columns="Unnamed: 0", inplace=True)
car_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.89,24.47,32.64,41.74,13.93,40.04,18.41,47.11,29.93,34.97,...,37.83,33.40,27.60,18.46,22.83,33.41,39.37,33.51,22.29,27.71
1,24.47,10.95,54.02,63.13,35.47,38.76,38.57,46.09,51.84,56.88,...,58.31,55.28,49.66,35.14,37.50,53.46,60.10,55.55,41.05,42.53
2,32.64,54.02,0.00,10.98,34.77,68.11,33.25,75.18,5.12,7.74,...,25.73,15.75,13.61,40.92,47.19,32.35,34.44,8.93,38.64,46.47
3,41.74,63.13,10.98,1.48,43.87,77.21,42.35,84.27,13.51,9.90,...,29.38,21.38,21.94,50.02,56.29,36.39,38.27,12.86,47.47,52.88
4,13.93,35.47,34.77,43.87,1.68,40.83,5.84,47.90,31.59,36.56,...,28.69,29.81,26.42,12.07,18.34,21.77,28.41,33.51,12.10,18.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,33.41,53.46,32.35,36.39,21.77,45.98,16.24,52.19,28.59,28.11,...,13.34,18.35,22.93,21.12,23.12,0.00,9.20,26.78,14.89,17.22
227,39.37,60.10,34.44,38.27,28.41,53.55,22.88,59.76,30.68,29.99,...,11.64,20.21,25.00,28.75,30.71,9.20,0.00,28.67,22.52,24.81
228,33.51,55.55,8.93,12.86,33.51,67.71,31.94,74.77,5.14,4.77,...,19.78,10.44,10.01,39.66,45.93,26.78,28.67,1.58,37.23,43.27
229,22.29,41.05,38.64,47.47,12.10,37.34,8.87,43.69,35.45,38.97,...,25.13,29.13,29.22,7.48,11.66,14.89,22.52,37.23,2.04,9.93


In [126]:
car_travel_cost_mat.drop(columns="Unnamed: 0", inplace=True)
car_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,58.06,192.97,244.02,300.88,127.07,290.28,155.04,334.43,227.06,258.58,...,276.42,248.76,212.52,155.36,182.72,248.83,286.05,249.47,179.32,213.17
1,192.97,108.46,377.66,434.54,261.71,282.24,281.05,328.06,363.98,395.50,...,404.46,385.48,350.38,259.64,274.38,374.12,415.61,387.20,296.59,305.79
2,244.02,377.66,40.00,108.64,257.34,465.70,247.82,509.86,71.98,88.35,...,200.82,138.46,125.08,295.75,334.94,242.22,255.28,95.81,281.50,330.42
3,300.88,434.54,108.64,49.27,314.20,522.56,304.68,566.71,124.44,101.89,...,223.62,173.60,177.15,352.60,391.80,267.40,279.20,120.37,336.67,370.48
4,127.07,261.71,257.34,314.20,50.50,295.20,76.52,339.36,237.46,268.51,...,219.31,226.32,205.15,115.43,154.62,176.06,217.54,249.45,115.62,156.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,248.83,374.12,242.22,267.40,176.06,327.36,141.53,366.22,218.70,215.66,...,123.39,154.70,183.30,172.02,184.51,40.00,97.52,207.39,133.07,147.63
227,286.05,415.61,255.28,279.20,217.54,374.66,183.01,413.51,231.76,227.44,...,112.74,166.30,196.27,219.71,231.96,97.52,40.00,219.18,180.76,195.08
228,249.47,387.20,95.81,120.37,249.45,463.16,239.61,507.31,72.10,69.80,...,163.61,105.26,102.53,287.85,327.04,207.39,219.18,49.86,272.71,310.46
229,179.32,296.59,281.50,336.67,115.62,273.35,95.41,313.09,261.56,283.56,...,197.07,222.08,222.64,86.77,112.86,133.07,180.76,272.71,52.73,102.09


In [127]:
motor_travel_time_mat.drop(columns="Unnamed: 0", inplace=True)
motor_travel_time_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,2.89,24.47,32.64,41.74,13.93,40.04,18.41,47.11,29.93,34.97,...,37.83,33.40,27.60,18.46,22.83,33.41,39.37,33.51,22.29,27.71
1,24.47,10.95,54.02,63.13,35.47,38.76,38.57,46.09,51.84,56.88,...,58.31,55.28,49.66,35.14,37.50,53.46,60.10,55.55,41.05,42.53
2,32.64,54.02,0.00,10.98,34.77,68.11,33.25,75.18,5.12,7.74,...,25.73,15.75,13.61,40.92,47.19,32.35,34.44,8.93,38.64,46.47
3,41.74,63.13,10.98,1.48,43.87,77.21,42.35,84.27,13.51,9.90,...,29.38,21.38,21.94,50.02,56.29,36.39,38.27,12.86,47.47,52.88
4,13.93,35.47,34.77,43.87,1.68,40.83,5.84,47.90,31.59,36.56,...,28.69,29.81,26.42,12.07,18.34,21.77,28.41,33.51,12.10,18.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,33.41,53.46,32.35,36.39,21.77,45.98,16.24,52.19,28.59,28.11,...,13.34,18.35,22.93,21.12,23.12,0.00,9.20,26.78,14.89,17.22
227,39.37,60.10,34.44,38.27,28.41,53.55,22.88,59.76,30.68,29.99,...,11.64,20.21,25.00,28.75,30.71,9.20,0.00,28.67,22.52,24.81
228,33.51,55.55,8.93,12.86,33.51,67.71,31.94,74.77,5.14,4.77,...,19.78,10.44,10.01,39.66,45.93,26.78,28.67,1.58,37.23,43.27
229,22.29,41.05,38.64,47.47,12.10,37.34,8.87,43.69,35.45,38.97,...,25.13,29.13,29.22,7.48,11.66,14.89,22.52,37.23,2.04,9.93


In [128]:
motor_travel_cost_mat.drop(columns="Unnamed: 0", inplace=True)
motor_travel_cost_mat

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,50.15,131.98,166.02,203.92,88.05,196.85,106.69,226.29,154.71,175.72,...,187.61,169.17,145.01,106.90,125.14,169.22,194.04,169.64,122.88,145.45
1,131.98,79.44,255.10,293.02,177.81,191.49,190.70,222.04,245.98,267.00,...,272.98,260.32,236.92,176.42,186.25,252.74,280.41,261.47,201.06,207.19
2,166.02,255.10,50.00,75.76,174.89,313.80,168.55,343.24,51.77,62.23,...,137.21,95.64,86.72,200.50,226.62,164.81,173.52,67.21,191.00,223.61
3,203.92,293.02,75.76,50.00,212.80,351.71,206.45,381.14,86.29,71.26,...,152.41,119.06,121.43,238.40,264.53,181.61,189.46,83.58,227.78,250.32
4,88.05,177.81,174.89,212.80,50.00,200.14,54.35,229.57,161.64,182.34,...,149.54,154.21,140.10,80.28,106.41,120.71,148.36,169.63,80.41,107.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,169.22,252.74,164.81,181.61,120.71,221.57,97.69,247.48,149.14,147.11,...,85.59,106.46,125.53,118.01,126.34,50.00,68.35,141.59,92.04,101.75
227,194.04,280.41,173.52,189.46,148.36,253.10,125.34,279.01,157.84,154.96,...,78.49,114.20,134.18,149.81,157.98,68.35,50.00,149.45,123.84,133.39
228,169.64,261.47,67.21,83.58,169.63,312.11,163.07,341.54,52.45,52.99,...,112.40,73.51,71.69,195.24,221.36,141.59,149.45,50.00,185.14,210.31
229,122.88,201.06,191.00,227.78,80.41,185.57,66.94,212.05,177.71,192.38,...,134.71,151.39,151.76,61.18,78.57,92.04,123.84,185.14,50.00,71.39


In [143]:
r = no_of_nodes
c = no_of_nodes
 
utility_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = bus_travel_time_mat[str(i)][j]
        travel_cost = bus_travel_cost_mat[str(i)][j]
        utility = U_bus(travel_time, travel_cost)
        utility_mat_bus[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_bus}')

df_utility_mat_bus = pd.DataFrame(utility_mat_bus)
df_utility_mat_bus

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,-0.1864,-2.1195,-2.7133,-3.6305,-0.9601,-3.7111,-1.5783,-4.6172,-2.4645,-3.0673,...,-14.9098,-2.9920,-5.8444,-1.3473,-1.7897,-5.1059,-3.3723,-2.9090,-1.7066,-2.0926
1,-2.1195,-8.6202,-4.9670,-5.8838,-3.0015,-3.7841,-3.7847,-4.6910,-4.7182,-5.3206,...,-14.9098,-5.2269,-7.4681,-3.0778,-3.2902,-7.3276,-5.4946,-5.1606,-3.6710,-3.7489
2,-2.7133,-4.9670,-0.0092,-0.7410,-3.5601,-6.3735,-2.8879,-7.2792,-0.3161,-0.5308,...,-14.9098,-1.0912,-4.6835,-3.6427,-4.2601,-3.0199,-2.9515,-0.5986,-3.4307,-4.0413
3,-3.6305,-5.8838,-0.7410,-0.1792,-4.4606,-7.2902,-3.7878,-8.1964,-0.9025,-0.6395,...,-14.9098,-1.6824,-5.2927,-4.5426,-5.0691,-3.3749,-3.2898,-1.0406,-4.2002,-4.8445
4,-0.9601,-3.0015,-3.5601,-4.4606,-0.1774,-3.3729,-1.0770,-4.2796,-3.3008,-3.8204,...,-14.9098,-3.2082,-6.3042,-0.6552,-1.3016,-4.6094,-2.4976,-3.7326,-0.9649,-1.3520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-5.1059,-7.3276,-3.0199,-3.3749,-4.6094,-7.3213,-3.9167,-8.2281,-2.6633,-2.6654,...,-14.9098,-2.0511,-5.8060,-4.5552,-4.9098,-0.0092,-2.4982,-2.9784,-4.1287,-4.4143
227,-3.3723,-5.4946,-2.9515,-3.2898,-2.4976,-5.1280,-1.8050,-5.8985,-2.5955,-2.5813,...,-14.9098,-1.8870,-5.6330,-2.4434,-2.4519,-2.4982,-0.0092,-2.9094,-1.8397,-1.9564
228,-2.9090,-5.1606,-0.5986,-1.0406,-3.7326,-6.5550,-3.0336,-7.4612,-0.3665,-0.5589,...,-14.9098,-1.0554,-4.6815,-3.7894,-4.4064,-2.9784,-2.9094,-0.2266,-3.5764,-4.1876
229,-1.7066,-3.6710,-3.4307,-4.2002,-0.9649,-3.2939,-0.6007,-4.1665,-3.1503,-3.4596,...,-14.9098,-2.7280,-6.0407,-0.6146,-0.6662,-4.1287,-1.8397,-3.5764,-0.1600,-0.5155


In [145]:
utility_mat_car = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = car_travel_time_mat[str(i)][j]
        travel_cost = car_travel_cost_mat[str(i)][j]
        utility = U_car(travel_time, travel_cost)
        utility_mat_car[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_car}')

df_utility_mat_car = pd.DataFrame(utility_mat_car)
df_utility_mat_car

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,-2.3623,-9.5468,-12.2656,-15.2940,-6.0374,-14.7292,-7.5273,-17.0808,-11.3626,-13.0411,...,-13.9915,-12.5182,-10.5881,-7.5442,-9.0009,-12.5218,-14.5043,-12.5558,-8.8201,-10.6230
1,-9.5468,-5.0461,-19.3827,-22.4125,-13.2077,-14.3014,-14.2380,-16.7415,-18.6547,-20.3332,...,-20.8101,-19.7997,-17.9302,-13.0976,-13.8827,-19.1946,-21.4043,-19.8910,-15.0653,-15.5557
2,-12.2656,-19.3827,-1.4004,-5.0558,-12.9749,-24.0716,-12.4681,-26.4237,-3.1038,-3.9756,...,-9.9651,-6.6438,-5.9313,-15.0208,-17.1079,-12.1697,-12.8652,-4.3727,-14.2619,-16.8674
3,-15.2940,-22.4125,-5.0558,-1.8939,-16.0033,-27.1000,-15.4965,-29.4510,-5.8974,-4.6963,...,-11.1794,-8.5157,-8.7043,-18.0487,-20.1362,-13.5113,-14.1393,-5.6807,-17.2002,-19.0009
4,-6.0374,-13.2077,-12.9749,-16.0033,-1.9596,-14.9918,-3.3452,-17.3435,-11.9163,-13.5700,...,-10.9499,-11.3231,-10.1955,-5.4176,-7.5047,-8.6465,-10.8558,-12.5549,-5.4277,-7.6089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-12.5218,-19.1946,-12.1697,-13.5113,-8.6465,-16.7044,-6.8073,-18.7735,-10.9173,-10.7557,...,-5.8413,-7.5088,-9.0322,-8.4312,-9.0964,-1.4004,-4.4635,-10.3149,-6.3569,-7.1324
227,-14.5043,-21.4043,-12.8652,-14.1393,-10.8558,-19.2235,-9.0166,-21.2922,-11.6128,-11.3828,...,-5.2744,-8.1268,-9.7226,-10.9709,-11.6233,-4.4635,-1.4004,-10.9430,-8.8967,-9.6593
228,-12.5558,-19.8910,-4.3727,-5.6807,-12.5549,-23.9368,-12.0311,-26.2878,-3.1102,-2.9876,...,-7.9836,-4.8758,-4.7309,-14.6003,-16.6874,-10.3149,-10.9430,-1.9257,-13.7936,-15.8040
229,-8.8201,-15.0653,-14.2619,-17.2002,-5.4277,-13.8281,-4.3516,-15.9436,-13.2000,-14.3716,...,-9.7654,-11.0972,-11.1271,-3.8910,-5.2808,-6.3569,-8.8967,-13.7936,-2.0786,-4.7069


In [146]:
utility_mat_motor = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        travel_time = motor_travel_time_mat[str(i)][j]
        travel_cost = motor_travel_cost_mat[str(i)][j]
        utility = U_motor(travel_time, travel_cost)
        utility_mat_motor[i][j] = round(utility, 4)

#print(f'travel time matrix of dimension {r} x {c} is {utility_mat_motor}')

df_utility_mat_motor = pd.DataFrame(utility_mat_motor)
df_utility_mat_motor

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,-1.1304,-5.9981,-7.9745,-10.1752,-3.4476,-9.7645,-4.5303,-11.4741,-7.3181,-8.5378,...,-9.2285,-8.1576,-6.7548,-4.5424,-5.6010,-8.1604,-9.6017,-8.1847,-5.4699,-6.7806
1,-5.9981,-2.8929,-13.1466,-15.3488,-8.6590,-9.4537,-9.4078,-11.2274,-12.6176,-13.8378,...,-14.1846,-13.4501,-12.0912,-8.5785,-9.1493,-13.0100,-14.6165,-13.5165,-10.0090,-10.3654
2,-7.9745,-13.1466,-0.9503,-2.7340,-8.4895,-16.5549,-8.1215,-18.2645,-1.3350,-1.9489,...,-6.3020,-3.8882,-3.3704,-9.9767,-11.4933,-7.9042,-8.4099,-2.2377,-9.4251,-11.3187
3,-10.1752,-15.3488,-2.7340,-1.0391,-10.6907,-18.7561,-10.3223,-20.4646,-3.3456,-2.4728,...,-7.1846,-5.2490,-5.3861,-12.1775,-13.6945,-8.8801,-9.3357,-3.1883,-11.5608,-12.8695
4,-3.4476,-8.6590,-8.4895,-10.6907,-1.0512,-9.9556,-1.4909,-11.6647,-7.7202,-8.9222,...,-7.0179,-7.2890,-6.4696,-2.9968,-4.5138,-5.3440,-6.9496,-8.1843,-3.0043,-4.5896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-8.1604,-13.0100,-7.9042,-8.8801,-5.3440,-11.2002,-4.0071,-12.7041,-6.9945,-6.8770,...,-3.3048,-4.5166,-5.6240,-5.1871,-5.6708,-0.9503,-2.3037,-6.5562,-3.6794,-4.2432
227,-9.6017,-14.6165,-8.4099,-9.3357,-6.9496,-13.0311,-5.6127,-14.5349,-7.4997,-7.3326,...,-2.8928,-4.9662,-6.1259,-7.0333,-7.5076,-2.3037,-0.9503,-7.0128,-5.5257,-6.0800
228,-8.1847,-13.5165,-2.2377,-3.1883,-8.1843,-16.4571,-7.8037,-18.1656,-1.3659,-1.3672,...,-4.8618,-2.6034,-2.4981,-9.6714,-11.1881,-6.5562,-7.0128,-1.0452,-9.0847,-10.5460
229,-5.4699,-10.0090,-9.4251,-11.5608,-3.0043,-9.1100,-2.2223,-10.6472,-8.6535,-9.5052,...,-6.1568,-7.1251,-7.1467,-1.8875,-2.8975,-3.6794,-5.5257,-9.0847,-1.0728,-2.4802


#### 9.2 Probability Matrices

In [147]:
def P_car(u_car, u_bus, u_motor):
    p_car = np.exp(u_car) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_car

def P_bus(u_car, u_bus, u_motor):
    p_bus = np.exp(u_bus) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_bus

def P_motor(u_car, u_bus, u_motor):
    p_motor = np.exp(u_motor) / (np.exp(u_car) + np.exp(u_bus) + np.exp(u_motor) )
    return p_motor

In [148]:
r = no_of_nodes
c = no_of_nodes
 
probability_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_bus(u_car, u_bus, u_motor)
        probability_mat_bus[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_bus}')

df_probability_mat_bus = pd.DataFrame(probability_mat_bus)
df_probability_mat_bus

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,0.6655,0.9792,0.9948,0.9986,0.9180,0.9976,0.9480,0.9989,0.9921,0.9958,...,0.0034,0.9943,0.7087,0.9588,0.9777,0.9544,0.9980,0.9948,0.9765,0.9907
1,0.9792,0.0029,0.9997,0.9999,0.9965,0.9965,0.9964,0.9985,0.9996,0.9998,...,0.3260,0.9997,0.9902,0.9959,0.9971,0.9966,0.9999,0.9998,0.9982,0.9987
2,0.9948,0.9997,0.6101,0.8698,0.9927,1.0000,0.9946,1.0000,0.7030,0.7849,...,0.0002,0.9391,0.1998,0.9982,0.9993,0.9924,0.9957,0.8216,0.9975,0.9993
3,0.9986,0.9999,0.8698,0.6237,0.9980,1.0000,0.9985,1.0000,0.9144,0.8495,...,0.0004,0.9715,0.5143,0.9995,0.9998,0.9959,0.9976,0.8878,0.9994,0.9997
4,0.9180,0.9965,0.9927,0.9980,0.6307,0.9986,0.5667,0.9994,0.9879,0.9939,...,0.0004,0.9831,0.5353,0.9052,0.9594,0.6678,0.9883,0.9883,0.8759,0.9605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.9544,0.9966,0.9924,0.9959,0.6678,0.9797,0.5078,0.9887,0.9868,0.9851,...,0.0000,0.9181,0.4466,0.6442,0.6746,0.6101,0.4247,0.9722,0.3738,0.4439
227,0.9980,0.9999,0.9957,0.9976,0.9883,0.9996,0.9776,0.9998,0.9925,0.9913,...,0.0000,0.9542,0.6144,0.9898,0.9936,0.4247,0.6101,0.9834,0.9747,0.9836
228,0.9948,0.9998,0.8216,0.8878,0.9883,0.9999,0.9915,1.0000,0.6981,0.6520,...,0.0000,0.8100,0.0924,0.9972,0.9989,0.9722,0.9834,0.6158,0.9959,0.9983
229,0.9765,0.9982,0.9975,0.9994,0.8759,0.9970,0.8189,0.9985,0.9959,0.9976,...,0.0002,0.9876,0.7479,0.7589,0.8950,0.3738,0.9747,0.9959,0.6459,0.8656


In [149]:
probability_mat_car = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_car(u_car, u_bus, u_motor)
        probability_mat_car[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_car}')

df_probability_mat_car = pd.DataFrame(probability_mat_car)
df_probability_mat_car

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,0.0755,0.0006,0.0001,0.0000,0.0057,0.0000,0.0025,0.0,0.0001,0.0000,...,0.0084,0.0001,0.0062,0.0020,0.0007,0.0006,0.0000,0.0001,0.0008,0.0002
1,0.0006,0.1037,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0009,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.0001,0.0000,0.1518,0.0116,0.0001,0.0000,0.0001,0.0,0.0433,0.0250,...,0.0250,0.0036,0.0574,0.0000,0.0000,0.0001,0.0000,0.0189,0.0000,0.0000
3,0.0000,0.0000,0.0116,0.1123,0.0000,0.0000,0.0000,0.0,0.0062,0.0147,...,0.0181,0.0010,0.0170,0.0000,0.0000,0.0000,0.0000,0.0086,0.0000,0.0000
4,0.0057,0.0000,0.0001,0.0000,0.1061,0.0000,0.0587,0.0,0.0002,0.0001,...,0.0192,0.0003,0.0109,0.0077,0.0019,0.0118,0.0002,0.0001,0.0101,0.0018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.0006,0.0000,0.0001,0.0000,0.0118,0.0001,0.0282,0.0,0.0003,0.0003,...,0.0733,0.0039,0.0177,0.0134,0.0103,0.1518,0.0595,0.0006,0.0403,0.0293
227,0.0000,0.0000,0.0000,0.0000,0.0002,0.0000,0.0007,0.0,0.0001,0.0001,...,0.0846,0.0019,0.0103,0.0002,0.0001,0.0595,0.1518,0.0003,0.0008,0.0004
228,0.0001,0.0000,0.0189,0.0086,0.0001,0.0000,0.0001,0.0,0.0449,0.0575,...,0.0422,0.0178,0.0879,0.0000,0.0000,0.0006,0.0003,0.1126,0.0000,0.0000
229,0.0008,0.0000,0.0000,0.0000,0.0101,0.0000,0.0192,0.0,0.0000,0.0000,...,0.0264,0.0002,0.0046,0.0287,0.0089,0.0403,0.0008,0.0000,0.0948,0.0131


In [150]:
probability_mat_motor = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        u_car = df_utility_mat_car[i][j]
        u_bus = df_utility_mat_bus[i][j]
        u_motor = df_utility_mat_motor[i][j]
        prob = P_motor(u_car, u_bus, u_motor)
        probability_mat_motor[i][j] = round(prob, 4)

#print(f'travel time matrix of dimension {r} x {c} is {probability_mat_bus}')

df_probability_mat_motor = pd.DataFrame(probability_mat_motor)
df_probability_mat_motor

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,0.2589,0.0202,0.0052,0.0014,0.0763,0.0023,0.0495,0.0011,0.0077,0.0042,...,0.9882,0.0057,0.2851,0.0393,0.0216,0.0450,0.0020,0.0051,0.0227,0.0091
1,0.0202,0.8934,0.0003,0.0001,0.0035,0.0034,0.0036,0.0014,0.0004,0.0002,...,0.6731,0.0003,0.0097,0.0041,0.0028,0.0034,0.0001,0.0002,0.0018,0.0013
2,0.0052,0.0003,0.2381,0.1185,0.0072,0.0000,0.0053,0.0000,0.2538,0.1901,...,0.9748,0.0573,0.7428,0.0018,0.0007,0.0075,0.0042,0.1595,0.0025,0.0007
3,0.0014,0.0001,0.1185,0.2640,0.0020,0.0000,0.0015,0.0000,0.0794,0.1358,...,0.9815,0.0275,0.4687,0.0005,0.0002,0.0040,0.0024,0.1037,0.0006,0.0003
4,0.0763,0.0035,0.0072,0.0020,0.2632,0.0014,0.3746,0.0006,0.0119,0.0060,...,0.9804,0.0166,0.4537,0.0871,0.0386,0.3204,0.0115,0.0115,0.1140,0.0377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.0450,0.0034,0.0075,0.0040,0.3204,0.0203,0.4639,0.0113,0.0130,0.0146,...,0.9267,0.0780,0.5357,0.3424,0.3152,0.2381,0.5158,0.0272,0.5859,0.5268
227,0.0020,0.0001,0.0042,0.0024,0.0115,0.0004,0.0217,0.0002,0.0074,0.0086,...,0.9154,0.0439,0.3753,0.0100,0.0063,0.5158,0.2381,0.0162,0.0244,0.0159
228,0.0051,0.0002,0.1595,0.1037,0.0115,0.0001,0.0084,0.0000,0.2570,0.2905,...,0.9577,0.1723,0.8197,0.0028,0.0011,0.0272,0.0162,0.2716,0.0040,0.0017
229,0.0227,0.0018,0.0025,0.0006,0.1140,0.0030,0.1618,0.0015,0.0041,0.0024,...,0.9735,0.0122,0.2475,0.2125,0.0961,0.5859,0.0244,0.0040,0.2593,0.1213


#### 10. Modal Share Matrix for Bus

In [151]:
# Import the Readjusted Final Trip Distribution from Matlab 
final_trip_dsbn = pd.read_csv("final_trip_distribution_mat.csv", header=None)

r = no_of_nodes
c = no_of_nodes

for i in range(0,r):
    for j in range(0,c):
        a = final_trip_dsbn[i][j]
        final_trip_dsbn.loc[i,j] = int(a)
        
final_trip_dsbn

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,45.0,6.0,5.0,18.0,0.0,4.0,0.0,10.0,1.0,...,13.0,10.0,22.0,4.0,7.0,10.0,32.0,0.0,4.0,24.0
2,0.0,6.0,36.0,31.0,96.0,0.0,1.0,0.0,45.0,0.0,...,2.0,6.0,8.0,21.0,10.0,38.0,17.0,0.0,2.0,16.0
3,0.0,5.0,31.0,138.0,186.0,0.0,2.0,0.0,80.0,0.0,...,2.0,8.0,7.0,28.0,15.0,35.0,14.0,1.0,1.0,13.0
4,0.0,18.0,96.0,186.0,123.0,0.0,1.0,0.0,0.0,0.0,...,1.0,5.0,6.0,28.0,10.0,27.0,12.0,0.0,1.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.0,10.0,38.0,35.0,27.0,0.0,4.0,0.0,40.0,0.0,...,11.0,6.0,19.0,8.0,19.0,0.0,7.0,0.0,0.0,6.0
227,0.0,32.0,17.0,14.0,12.0,0.0,6.0,0.0,17.0,0.0,...,35.0,7.0,87.0,5.0,14.0,7.0,161.0,0.0,7.0,61.0
228,0.0,0.0,0.0,1.0,0.0,1.0,3.0,44.0,1.0,99.0,...,2.0,1.0,0.0,0.0,2.0,0.0,0.0,551.0,0.0,0.0
229,0.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,4.0,0.0,7.0,1.0,1.0,0.0,7.0,0.0,29.0,48.0


In [152]:
modal_share_mat_bus = [[0 for _ in range(c)] for _ in range(r)]

for i in range(0,r):
    for j in range(0,c):
        trip_bus = final_trip_dsbn[i][j]
        prob_bus = df_probability_mat_bus[i][j]
        modal_share_mat_bus[i][j] = round(trip_bus*prob_bus, 0)

#print(f'travel time matrix of dimension {r} x {c} is {modal_share_mat_bus}')

df_modal_share_mat_bus = pd.DataFrame(modal_share_mat_bus)
df_modal_share_mat_bus

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,6.0,5.0,18.0,0.0,4.0,0.0,10.0,1.0,...,4.0,10.0,22.0,4.0,7.0,10.0,32.0,0.0,4.0,24.0
2,0.0,6.0,22.0,27.0,95.0,0.0,1.0,0.0,32.0,0.0,...,0.0,6.0,2.0,21.0,10.0,38.0,17.0,0.0,2.0,16.0
3,0.0,5.0,27.0,86.0,186.0,0.0,2.0,0.0,73.0,0.0,...,0.0,8.0,4.0,28.0,15.0,35.0,14.0,1.0,1.0,13.0
4,0.0,18.0,95.0,186.0,78.0,0.0,1.0,0.0,0.0,0.0,...,0.0,5.0,3.0,25.0,10.0,18.0,12.0,0.0,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0.0,10.0,38.0,35.0,18.0,0.0,2.0,0.0,39.0,0.0,...,0.0,6.0,8.0,5.0,13.0,0.0,3.0,0.0,0.0,3.0
227,0.0,32.0,17.0,14.0,12.0,0.0,6.0,0.0,17.0,0.0,...,0.0,7.0,53.0,5.0,14.0,3.0,98.0,0.0,7.0,60.0
228,0.0,0.0,0.0,1.0,0.0,1.0,3.0,44.0,1.0,65.0,...,0.0,1.0,0.0,0.0,2.0,0.0,0.0,339.0,0.0,0.0
229,0.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,5.0,1.0,1.0,0.0,7.0,0.0,19.0,42.0


In [153]:
# Export as .csv
df_modal_share_mat_bus.to_csv("df_modal_share_mat_bus.csv")

In [159]:
df_modal_share_mat_bus = pd.read_csv("df_modal_share_mat_bus.csv")
df_modal_share_mat_bus.drop(columns="Unnamed: 0", inplace=True)
df_modal_share_mat_bus.head()

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,6.0,5.0,18.0,0.0,4.0,0.0,10.0,1.0,...,4.0,10.0,22.0,4.0,7.0,10.0,32.0,0.0,4.0,24.0
2,0.0,6.0,22.0,27.0,95.0,0.0,1.0,0.0,32.0,0.0,...,0.0,6.0,2.0,21.0,10.0,38.0,17.0,0.0,2.0,16.0
3,0.0,5.0,27.0,86.0,186.0,0.0,2.0,0.0,73.0,0.0,...,0.0,8.0,4.0,28.0,15.0,35.0,14.0,1.0,1.0,13.0
4,0.0,18.0,95.0,186.0,78.0,0.0,1.0,0.0,0.0,0.0,...,0.0,5.0,3.0,25.0,10.0,18.0,12.0,0.0,1.0,12.0


In [161]:
# Export to Matlab
import scipy.io
final_array_demand_mat = np.array(df_modal_share_mat_bus)

final_demand_arr = final_array_demand_mat
final_demand_arr = final_demand_arr.reshape((231,231))
# save to .mat file with variable final_demand_mat
scipy.io.savemat('final_demand.mat', {'array': final_demand_arr})